In [40]:
using Pkg
Pkg.activate(".")
Pkg.add("Plots")
Pkg.add("Distributed")
Pkg.add("BenchmarkTools")
Pkg.add("CUDA")
using Plots
using Distributed
using BenchmarkTools
using CUDA

  Activating project at `c:\School\BS Thesis\src`
   Resolving package versions...
  No Changes to `C:\School\BS Thesis\src\Project.toml`
  No Changes to `C:\School\BS Thesis\src\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\School\BS Thesis\src\Project.toml`
  No Changes to `C:\School\BS Thesis\src\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\School\BS Thesis\src\Project.toml`
  No Changes to `C:\School\BS Thesis\src\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\School\BS Thesis\src\Project.toml`
  No Changes to `C:\School\BS Thesis\src\Manifest.toml`


In [28]:
function initialize_grid(size)
    grid = zeros(Int64,size,size)
    grid[2:size-1,2:size-1] = rand([0,1],size-2,size-2)
    return grid
end

initialize_grid (generic function with 1 method)

In [ ]:
function generate_next_generation(grid,size)
    next_generation = zeros(Int64,size,size)
    for y in 2:size-1, x in 2:size-1
        temp_grid = grid[y-1:y+1,x-1:x+1]
        temp_grid[2,2] = 0 
        temp_sum = sum(temp_grid)

        if temp_sum < 2 || temp_sum > 3             # if less than 2 or more than 3
            next_generation[y,x] = 0

        elseif temp_sum == 3
            next_generation[y,x] = 1                # if 3 neighbors

        elseif temp_sum == 2 && grid[y,x] == 1      # if 2 neighbors and alive
            next_generation == 1

        end

    end

    return next_generation

end

In [ ]:
function generate_animation(grid_collection,filename)
    Ani = Animation()
    println("generating animation")


    for generation_number in range(1,length(grid_collection))
        if generation_number%(length(grid_collection)//10) == 0
            println("Animating frame $(generation_number)/$(length(grid_collection))")
        end
        frame(Ani,
            heatmap(grid_collection[generation_number],
            plot_title = "Conway's Game of Life - Generation $(generation_number)",
            plot_titlefontsize = 12,
            yflip=true,
            aspect_ratio=:equal,
            xlims=(0.5,size+0.5),
            showaxis = false, #Removes the axes' black lines
            axis=([], false), #Removes the numbrs on the axis
            ylims=(0.5,size+0.5),
            dpi=300,
            cbar=false,
            c = palette(:grays,rev=true)
            )
        
        )
    end

    return gif(Ani,filename,fps=2)
end

LoadError: LoadError: UndefVarError: @parallel not defined
in expression starting at In[37]:7

In [35]:
## Tried using @distributed in the generate_next_generation function

In [43]:
@benchmark begin
size = 500
max_generations = 25
grid_collection = []

grid = initialize_grid(size)


for generation in 1:max_generations
    if generation%(max_generations//10)==0
        #println("generating generation: $(generation)")
    end
    push!(grid_collection,grid)
    grid = generate_next_generation(grid,size)
end
end

#generate_animation(grid_collection,"C:/School/BS Thesis/src/game-of-life.gif")


┌ Warning: Performing scalar indexing on task Task (runnable) @0x000000000bb0f080.
│ Invocation of setindex! resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArraysCore C:\Users\ioakimsy\.julia\packages\GPUArraysCore\lojQM\src\GPUArraysCore.jl:90


In [33]:
@benchmark begin
    size = 500
    max_generations = 25
    grid_collection = []
    
    grid = initialize_grid(size)
    
    
    for generation in 1:max_generations
        if generation%(max_generations//10)==0
            #println("generating generation: $(generation)")
        end
        push!(grid_collection,grid)
        grid = generate_next_generation(grid,size)
    end
    end
    
    #generate_animation(grid_collection,"C:/School/BS Thesis/src/game-of-life.gif")
    

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  978.549 ms …    1.553 s  ┊ GC (min … max):  6.32% … 37.89%
 Time  (median):        1.235 s               ┊ GC (median):     8.60%
 Time  (mean ± σ):      1.250 s ± 252.572 ms  ┊ GC (mean ± σ):  17.25% ± 15.05%

  █              █                       █                    █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  979 ms           Histogram: frequency by time          1.55 s <

 Memory estimate: 808.33 MiB, allocs estimate: 6200158.

In [41]:
CUDA.versioninfo()

CUDA toolkit 11.7, artifact installation
NVIDIA driver 526.98.0, for CUDA 12.0
CUDA driver 12.0

Libraries: 
- CUBLAS: 11.10.1
- CURAND: 10.2.10
- CUFFT: 10.7.1
- CUSOLVER: 11.3.5
- CUSPARSE: 11.7.3
- CUPTI: 17.0.0
- NVML: 11.0.0+526.98


- CUDNN: 8.30.2 (for CUDA 11.5.0)


- CUTENSOR: 1.4.0 (for CUDA 11.5.0)

Toolchain:
- Julia: 1.8.2
- LLVM: 13.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: NVIDIA GeForce GTX 1650 Ti with Max-Q Design (sm_75, 2.514 GiB / 4.000 GiB available)
